In [2]:
import json
import numpy as np
from tensorflow.keras.applications.resnet50 import ResNet50, preprocess_input
import shap
from PIL import Image
import time

In [3]:
img_path = "../data/figure_data/dog_and_cat.jpg"
img = Image.open(img_path).convert('RGB').resize((224, 224))
img_np = np.array(img)
img_batch = np.expand_dims(img_np, axis=0)
img_preprocessed = preprocess_input(img_batch.copy())

X = img_preprocessed

In [4]:
url = "https://s3.amazonaws.com/deep-learning-models/image-models/imagenet_class_index.json"
with open(shap.datasets.cache(url)) as file:
    class_names = [v[1] for v in json.load(file).values()]

In [ ]:
start = time.time()
def f(x):
    tmp = x.copy()
    preprocess_input(tmp)
    return model(tmp)


# define a masker that is used to mask out partitions of the input image.
masker = shap.maskers.Image("inpaint_telea", X[0].shape)

# create an explainer with model and image masker
explainer = shap.Explainer(f, masker, output_names=class_names)

# here we explain two images using 500 evaluations of the underlying model to estimate the SHAP values
#X_orig = X[[1]]  # Use original RGB values, not preprocessed
#X_display = X_orig / 255.0  # if original image is in [0, 255]
shap_values = explainer(X, max_evals=100, batch_size=50, outputs=shap.Explanation.argsort.flip[:1])

In [ ]:
img_raw = Image.open(img_path).convert("RGB").resize((224, 224))
X_orig = np.array(img_raw)
X_display = X_orig / 255.0

# Make it a batch of 1 image with shape (1, 224, 224, 3)
X_display = np.array([X_display])

shap.image_plot(shap_values, X_display)
end = time.time()
duration = end - start
print(duration)